
# NAMD Protein-ligand complex MD Setup tutorial using Jupyter Notebook

This notebook is wrritten by quantaosun@gmail.com in Shanghai, China, 2021. 
This notebook took PDB ID 7L10, the protease of SARS-CoV-2 with an inhibitor, as an example, relative work was firstly published on ACS Cent. Sci. 2021, 7, 3, 467–475, by William L. Jorgensen, Yale University.

**The overall procedure is as below**
0.   PDB structure check and fix with third-party tools or web server.
1.   Installation of Anaconda,pymol, vmd, and openmm, pdbfixer
2.   PDB ID provided by user
3.   Separation of ligand and protein
4.   Topology generation for ligand using LigParGen web server
5.   Combine new ligand pdb and original protein, mannually renumber ligand atom
6.   Generation of ionized.pdb and ionized.psf by VMD, with Charmm force field.
7.   Generation of conf file for NAMD simulation. 
8.   NAMD simulation 


# Download Github repository and create conda env

In [ ]:
$ git clone https://github.com/quantaosun/NAMD-MD
$ cd NAMD-MD
$ conda create -n NAMD-MD python=3.7
$ conda activate NAMD-MD
$ (NAMD-MD) conda install jupyter
$ (NAMD-MD) jupyter notebook
$ # If you have problem opening jupyter notebook
$ # conda remove jupyter
$ # sudo apt isntall jupyter
$ # jupyter notebook

# Starting Jupyter notebook and install dependencies, this may take a while

In [7]:
! yes | conda install -c conda-forge vmd &> /dev/null
#@title Install VMD
!yes | conda install -c conda-forge pymol-open-source &> /dev/null
#install openmm
!yes | conda install -c conda-forge openmm &> /dev/null
#install pdbfixer
!yes | conda install -c conda-forge pdbfixer &> /dev/null
#install open babel
!yes|conda install -c openbabel openbabel
#install visualization
! pip install py3Dmol &> /dev/null
#install rdkit
!yes | conda install -c rdkit rdkit &> /dev/null

Solving environment: done

# All requested packages already installed.

yes: standard output: Broken pipe


# Use "conda env list" find your system path

In [8]:
import sys
sys.path.append('/home/sqt/anaconda3/envs/NAMD-MD/lib/python3.7/site-packages')
#import rdkit for visualization purpose
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import DataStructs
from rdkit.Chem import RDConfig
from rdkit.Chem import rdBase
from rdkit.Chem import MolFromPDBFile

# Visually check if your protein has multiple chains or just a single one.

In [9]:
import py3Dmol
view = py3Dmol.view(query='pdb:7l10')
view.setStyle({'cartoon':{'color':'spectrum'}})
view

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Download PDB 

In [4]:
PDB_ID = "7L10???" #@param {type:"string"}
pdb = PDB_ID + ".pdb"
! wget https://files.rcsb.org/download/$pdb

--2021-11-30 20:48:43--  https://files.rcsb.org/download/7L10.pdb
Resolving files.rcsb.org (files.rcsb.org)... 132.249.210.134
Connecting to files.rcsb.org (files.rcsb.org)|132.249.210.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/octet-stream]
Saving to: ‘7L10.pdb’

7L10.pdb                [       <=>          ] 241.42K  7.09KB/s    in 29s     

2021-11-30 20:49:14 (8.46 KB/s) - ‘7L10.pdb’ saved [247212]



# (Optional) Skip pdbfixer if your protein has no missing loops

In [ ]:
#Only use this if your protein has a broken backbone
#!pdbfixer 7L10.pdb --keep-heterogens=none --add-atoms=heavy --ph=7.0 --replace-nonstandard --output=receptor.pdbfixer.pdb

# What is the name of your ligand? Check it out on RCSB PDB website !

In [10]:
v = py3Dmol.view(query='pdb:7L10',style={'cartoon':{'colorscheme':'ssPyMol'},'stick':{'radius':0.05}})
v.setStyle({'resn':'XEY'},{'stick':{}})
v.zoomTo({'resn':'XEY'})

You appear to be running in JupyterLab (or JavaScript failed to load for some other reason). You need to install the 3dmol extension: 
 jupyter labextension install jupyterlab_3dmol

# Split protein and ligand, input the PDB ID and Ligand ID.

In [14]:
ligand_name = "XEY????" #@param {type:"string"}
PDB_path = "7L10.pdb??????" #@param {type:"string"}
!grep ATOM '{PDB_path}' > protein_no_H.pdb
!grep HETATM '{PDB_path}' > ligand_1.pdb
!grep '{ligand_name}' ligand_1.pdb > ligand_no_H.pdb

# Add hydrogen to ligand

In [15]:
!obabel -ipdb ligand_no_H.pdb -opdb -O ligand_with_H.pdb -h

1 molecule converted


# Uplaod ligand_with_H.pdb to LigParGen to generate PDB,PRM and RTF three files.  http://zarbi.chem.yale.edu/ligpargen/index.html 

In [16]:
com_file = open('combine_protien_ligand.pml','w')
com_file.write('''
load protein_no_H.pdb
load XEY_51474E.pdb ???????????????
select all
save complex.pdb, all 
''')
com_file.close()
!pymol -c combine_protien_ligand.pml

 PyMOL(TM) Molecular Graphics System, Version 2.5.0.
 Copyright (c) Schrodinger, LLC.
 All Rights Reserved.
 
    Created by Warren L. DeLano, Ph.D. 
 
    PyMOL is user-supported open-source software.  Although some versions
    are freely available, PyMOL is not in the public domain.
 
    If PyMOL is helpful in your work or study, then please volunteer 
    support for our ongoing efforts to create open and affordable scientific
    software by purchasing a PyMOL Maintenance and/or Support subscription.

    More information can be found at "http://www.pymol.org".
 
    Enter "help" for a list of commands.
    Enter "help <command-name>" for information on a specific command.

 Hit ESC anytime to toggle between text and graphics.

 Detected 16 CPU cores.  Enabled multithreaded rendering.
PyMOL>load protein_no_H.pdb
 CmdLoad: "" loaded as "protein_no_H".
PyMOL>load XEY_51474E.pdb
 CmdLoad: "" loaded as "XEY_51474E".
PyMOL>select all
 Selector: selection "sele" defined with 2388 atoms

# Modify VMD script and run psfgen

In [17]:
com_file = open('psfgen.tcl','w')
com_file.write('''
#psfgen for a single-chained protein with a ligand bound
mol delete all
mol load pdb complex.pdb
set bad [atomselect top "resname ACE"]
if {[info exists bad]} {
set chainB [atomselect top "protein and not hydrogen and not resname ACE NME"]
set chainX [atomselect top "residuetype nothing and not resname ACE NME"]
set flag 1
} else {
set chainB [atomselect top "protein and not hydrogen"]
set chainX [atomselect top "residuetype nothing"]
set flag 0}
$chainB writepdb chainB.pdb
$chainX writepdb chainX.pdb
package require psfgen
topology top_all36_prot.rtf
topology XEY_51474E.rtf??????????????????
pdbalias HIS HSD
pdbalias atom SER HG HG1
pdbalias residue HIS HSE
pdbalias atom ILE CD1 CD
if {$flag == 1} {
segment B {
  first ACE
  last CT3
  pdb chainB.pdb
}
} else {
segment B {
  first NONE
  last NONE
  pdb chainB.pdb
}
}
segment X {
  first NONE
  last NONE
  pdb chainX.pdb
}
coordpdb chainB.pdb B
coordpdb chainX.pdb X
guesscoord
regenerate angles dihedrals
writepdb psf-complex.pdb
writepsf psf-complex.psf
exit
''')
com_file.close()
!vmd -dispdev text -e psfgen.tcl

/home/q/anaconda3/envs/NAMD-MD/lib/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /home/q/anaconda3/envs/NAMD-MD/lib/vmd_LINUXAMD64)
Info) VMD for LINUXAMD64, version 1.9.3 (November 30, 2016)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 16 CPUs detected.
Info)   CPU features: SSE2 AVX AVX2 FMA 
Info) Free system memory: 13GB (87%)
Info) No CUDA accelerator devices available.
Warning) Unable to open dynamic library '/home/q/anaconda3/envs/NAMD-MD/lib/plugins/LINUXAMD64/molfile/hoomdplugin.so'.
Warning) libexpat.so.0: failed to map segment from shared

# Solvate the Complex and add ions 

In [18]:
com_file = open('solvate.tcl','w')
com_file.write('''
package require solvate  
solvate psf-complex.psf psf-complex.pdb -t 15 -o complex_wb 
package require autoionize
autoionize -psf complex_wb.psf -pdb complex_wb.pdb -sc 0.15 -cation SOD -o ionized 
exit
''')
com_file.close()
!vmd -dispdev text -e solvate.tcl

/home/q/anaconda3/envs/NAMD-MD/lib/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /home/q/anaconda3/envs/NAMD-MD/lib/vmd_LINUXAMD64)
Info) VMD for LINUXAMD64, version 1.9.3 (November 30, 2016)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 16 CPUs detected.
Info)   CPU features: SSE2 AVX AVX2 FMA 
Info) Free system memory: 13GB (87%)
Info) No CUDA accelerator devices available.
Warning) Unable to open dynamic library '/home/q/anaconda3/envs/NAMD-MD/lib/plugins/LINUXAMD64/molfile/hoomdplugin.so'.
Warning) libexpat.so.0: failed to map segment from shared

Autoionize) Obtained positions for 110 ions.
Autoionize) Tagged 110 water molecules for deletion.
Autoionize) Deleted 110 water molecules.
Autoionize) Reading CHARMM topology file...
psfgen) reading topology file /home/q/anaconda3/envs/NAMD-MD/lib/plugins/noarch/tcl/readcharmmtop1.2/toppar_water_ions_namd.str

psfgen)  Toplogy and parameter information for water and ions.
psfgen) 
psfgen) reading topology from stream file
psfgen)  Topology for water and ions
psfgen) 
psfgen) 31  1
psfgen)  Parameters for water and ions
psfgen) 
psfgen) ERROR!  Failed to parse atom statement.  Line 152: ATOMS

psfgen) duplicate type key HT
psfgen) duplicate type key HX
psfgen) duplicate type key OT
psfgen) duplicate type key OX
psfgen) duplicate type key LIT
psfgen) duplicate type key SOD
psfgen) duplicate type key MG
psfgen) duplicate type key POT
psfgen) duplicate type key CAL
psfgen) duplicate type key RUB
psfgen) duplicate type key CES
psfgen) duplicate type key BAR
psfgen) duplicate type key ZN
psf

# (Optional, you can skip this) Make restrain on alpha carbon of protein.

In [23]:
com_file = open('restrain.tcl','w')
com_file.write('''
mol delete all
mol load pdb ionized.pdb
set all [atomselect top "all"]
set sel [atomselect top "protein and name CA"]
$all set beta 0
$sel set beta 1
$all writepdb restrained.pdb
exit
''')
com_file.close()
!vmd -dispdev text -e restrain.tcl

/home/q/anaconda3/envs/NAMD-MD/lib/vmd_LINUXAMD64: /lib/x86_64-linux-gnu/libGL.so.1: no version information available (required by /home/q/anaconda3/envs/NAMD-MD/lib/vmd_LINUXAMD64)
Info) VMD for LINUXAMD64, version 1.9.3 (November 30, 2016)
Info) http://www.ks.uiuc.edu/Research/vmd/                         
Info) Email questions and bug reports to vmd@ks.uiuc.edu           
Info) Please include this reference in published work using VMD:   
Info)    Humphrey, W., Dalke, A. and Schulten, K., `VMD - Visual   
Info)    Molecular Dynamics', J. Molec. Graphics 1996, 14.1, 33-38.
Info) -------------------------------------------------------------
Info) Multithreading available, 16 CPUs detected.
Info)   CPU features: SSE2 AVX AVX2 FMA 
Info) Free system memory: 13GB (86%)
Info) No CUDA accelerator devices available.
Warning) Unable to open dynamic library '/home/q/anaconda3/envs/NAMD-MD/lib/plugins/LINUXAMD64/molfile/hoomdplugin.so'.
Warning) libexpat.so.0: failed to map segment from shared

# Measure the water box dimensions

In [ ]:
com_file = open('measure.tcl','w')
com_file.write('''
mol delete all
mol load pdb ionized.pdb
set ubq [atomselect top all]
measure minmax $ubq   
measure center $ubq 
''')
com_file.close()
!vmd -dispdev text -e measure.tcl

# Modify NAMD configuration files

# Three parts in nvt.namd need modification, same for npt.namd

In [ ]:
# Periodic Boundary Conditions
cellBasisVector1 ? 0 0
cellBasisVector2 0 ? 0
cellBasisVector3 0 0 ?
cellOrigin ? ? ?
wrapAll on
# PME (for full-system periodic electrostatics)
PME yes
PMEGridSizeX ? ;# product of 2,3,5, slightly bigger than box vector
PMEGridSizeY ? ;# product of 2,3,5
PMEGridSizeZ ? ;# product of 2,3,5

In [ ]:
paraTypeCharmm on

parameters                        top_all36_prot.prm
parameters                        ???????????
parameters                        toppar_water_ions.mod.str

In [ ]:
run ???????? ;# 5000ps

# Run the simulation (This assume you have installed namd)

In [ ]:

!namd2 +p2 nvt.namd > nvt.log
#!namd2 +p2 npt.namd > npt.log
